In [1]:
#Черепов Арсений Би-17-2
#ссылка на репозиторий GitHub
#https://github.com/arseniycherepov/MachineLearning
#Подключаем библиотеки
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#Считываем данные с файлов
transactions = pd.read_csv('casc-resto.csv', sep=';', parse_dates = ['RKDate'], decimal=',')
clients = pd.read_csv('CASC_Constant.csv', sep =',')

In [3]:
#Выполняем очистку и подготовку данных к анализу
transactions.dropna()
transactions.drop_duplicates(keep=False, inplace=True)
clients.dropna()
clients.drop_duplicates(keep=False, inplace=True)
clients['Sex'] = clients['Sex'].astype('category').cat.codes
clients['Age'] = clients['Age'].apply(
    lambda x: int(clients['Age'].mean()) if x > 100 else x)

In [4]:
#Присваиваем стартовую и конечную дату переменным
date1 = datetime(2017, 7, 1)
date2 = datetime(2017, 12, 31)

In [5]:
#Для каждого клиента определим, совершал ли он визит в период между 2017-07-01 и 2017-12-31 включительно
def GetY(clients):
    return len(clients.loc[(clients['RKDate'] >= date1) & (clients['RKDate'] <= date2)]) > 0

#Расчитаем количество дней до последнего визита клиента перед зафиксированной датой 2017-07-01.
def Recency(clients):    
    OrdersBefore = clients.loc[clients['RKDate'] < date1]
    LastOrder = OrdersBefore['RKDate'].max()
    return pd.Timedelta(date1 - LastOrder).days

#Определим среднюю частоту походов клиентов на дату 2017-07-01.
def Frequency(clients):
    OrdersBefore = clients.loc[clients['RKDate'] < date1]
    return OrdersBefore.groupby([OrdersBefore['RKDate'].dt.year,
                                 OrdersBefore['RKDate'].dt.month]).size().mean()

#Рассчитаем средний чек клиента по всем покупкам до 2017-07-01.
def MonetaryValue(clients):
    OrdersBefore = clients.loc[clients['RKDate'] < date1]
    return OrdersBefore['SummAfterPointsUsage'].mean()

#Расчитаем функции  
def RunFunctions(clients):
    result = [{
        'Y': GetY(clients),
        'Recency': Recency(clients),
        'Frequency': Frequency(clients),
        'MonetaryValue': MonetaryValue(clients)
    }]
    return pd.DataFrame(data = result)


In [ ]:
#Вызовем итоговую функцию и притянем данные из второй таблицы
ClientResult = transactions.groupby(by='CustomerID').apply(lambda temp: RunFunctions(temp))
ClientResult.dropna()
ClientResult = ClientResult.join(clients[['CustomerId', 'Age', 'Sex']].set_index('CustomerId'), on = ['CustomerID'], how='right')

In [ ]:
#Выделим зависимую переменную
y = ClientResult['Y']
X = ClientResult.drop('Y', axis=1)
#Разделим данные на обучающую и тестовую выборку в соотношении 80 на 20
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=25, test_size=0.2)
#Обучим модель логистической регрессии на обучающей выборке.
Regression = LogisticRegression(random_state = 25, solver='lbfgs', max_iter=120)
Regression.fit(X_train, y_train)
#Предскажем вероятность прихода на тестовой выборке.
Prediction = Regression.predict(X_test)

In [ ]:
#Посчитаем метрики Precision и Recall на основании предсказаний на тестовой выборке и истинных значений зависимой переменной Y
print(classification_report(y_test, y_predict))